In [77]:
import pandas as pd
import numpy as np

m5 = []
m15 = []
m30 = []
h1 = []
h4 = []
d1 = []
quarter_hours = [":15", "45"]
four_hours = ["04:00", "08:00", "12:00", "16:00", "20:00"]
timeframes = [
    m5,
    # m15, m30, h1, h4, d1
]

timeframe_names = [
    "m5",
    # "m15", "m30", "h1", "h4", "d1"
]
symbols = [
    # "GBPUSD_2021_m1.csv",
    # "GBPUSD_2020_m1.csv",
    # "GBPUSD_2019_m1.csv",
    # "GBPJPY_2021_m1.csv",
    # "GBPJPY_2020_m1.csv",
    # "GBPJPY_2019_m1.csv",
    # "EURUSD_2021_m1.csv",
    # "EURUSD_2020_m1.csv",
    "EURUSD_2019_m1.csv",
]


def processSymbol(symbol):
    symbol_path = (
        "/media/bruno/f71fc4fd-abcd-4a4b-9fb1-6a8899041c1c/FX-Training-Data/ALL/"
        + str(symbol)
    )
    base_data = np.array(
        pd.read_csv(
            symbol_path,
            header=None,
        )
    )
    return base_data


In [78]:
def makeTimeframeData(base_data):
    # print("making timeframe data")

    for i in range(len(base_data)):

        if str(base_data[i][1]).endswith("0"):
            m5.append(i)

            if str(base_data[i][1]).endswith("00"):
                h1.append(i)
                m30.append(i)
                m15.append(i)
                if str(base_data[i][1]) in four_hours:
                    h4.append(i)
                    continue
                if str(base_data[i][1]) == "00:00":
                    d1.append(i)
                    h4.append(i)
                    continue
            elif str(base_data[i][1]).endswith("30"):
                m30.append(i)
                m15.append(i)
                continue

        elif str(base_data[i][1]).endswith("5"):
            m5.append(i)
            if str(base_data[i][1]).endswith("45") or str(base_data[i][1]).endswith(
                "15"
            ):
                m15.append(i)
            continue


In [79]:
class Row:
    def __init__(self, date, time, open_p, high, low, close, volume, bullish, bearish, neutral):
        self.date = date
        self.time = time
        self.open_p = float(open_p)
        self.high = float(high)
        self.low = float(low)
        self.close = float(close)
        self.volume = int(volume)
        self.bullish = int(bullish)
        self.bearish = int(bearish)
        self.neutral = int(neutral)

    def makeRecord(self):
        return list(
            [
                self.date,
                self.time,
                self.open_p,
                self.high,
                self.low,
                self.close,
                self.volume,
                self.bullish,
                self.bearish,
                self.neutral
            ]
        )


In [80]:
def getMax(base_data, column_index, start, stop):
    values = list()
    for i in range(start, stop):
        values.append(float(base_data[i][column_index]))
    return max(values)


def getMin(base_data, column_index, start, stop):
    values = list()
    for i in range(start, stop):
        values.append(float(base_data[i][column_index]))
    return min(values)


def getSum(base_data, column_index, start, stop):
    values = list()
    for i in range(start, stop):
        values.append(float(base_data[i][column_index]))
    return sum(values)


So now that you have lists of positions for each time interval, use those to recalculate OHLCVS(indices 2,3,4,5,6,7)
<!-- for each index listed in h1,h4,etc look for a corresponding element in base_data -->
<!-- open = same open as that index -->
<!-- close = close of index prior to interval index -->
<!-- high = highest value base_data[index][3] in range(curr_interval_index, next_interval_index) -->
<!-- low =  lowest value base_data[index][3] in range(curr_interval_index, next_interval_index)-->
<!-- volume = sum of base_data[index][6] in range(curr_interval_index, next_interval_index) -->
<!-- state = [0 if open==close, 1 if close>open, -1 if close<open] -->

In [81]:
def processCsv(base_data, timeframe, symbol):
    data = [["date", "time", "open", "high", "low", "close", "volume", "bullish", "bearish", "neutral"]]

    for i in range(len(timeframe)):
        # if i == 0:
        #     # data = [["date", "time", "open", "high", "low", "close", "volume", "state"]]
        #     print("starting afresh")
        if (i + 1) == len(timeframe):
            data = pd.DataFrame(data)
            newCsv_name = (
                "Sorted/"
                + symbol[:11]
                + "_"
                + str(timeframe_names[timeframes.index(timeframe)])
                + ".csv"
            )
            data.to_csv(newCsv_name, header=False)
            break

        row = Row(
            date=base_data[timeframe[i]][0],
            time=base_data[timeframe[i]][1],
            open_p=base_data[timeframe[i]][2],
            high=getMax(base_data, 3, timeframe[i], timeframe[i + 1]),
            low=getMin(base_data, 4, timeframe[i], timeframe[i + 1]),
            close=base_data[(timeframe[i + 1]) - 1][5],
            volume=getSum(base_data, 6, timeframe[i], timeframe[i + 1]),
            bullish = 0,
            bearish = 0,
            neutral = 0
        )
        setCandleState(row=row)
        data.append(row.makeRecord())


def setCandleState(row):
    if row.close > row.open_p:
        row.bullish = 1
    elif row.close < row.open_p:
        row.bearish = 1
    else:
        row.neutral = 1


In [82]:
def clrLists(timeframes):
    for tf in timeframes:
        tf.clear()


In [83]:
for sy in symbols:
    clrLists(timeframes=timeframes)
    base_data = processSymbol(sy)
    makeTimeframeData(base_data=base_data)
    for tf in timeframes:
        # print("length of base_data :{}, symbol:{}".format(str(len(base_data)), str(sy)))
        # print("length of timeframe_data : {}".format(str(len(tf))))
        processCsv(base_data=base_data, timeframe=tf, symbol=sy)


/tmp/ipykernel_2220/2644059365.py:40: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(
